# Práctica 4: Transformers y modelos contextuales para el Procesamiento del Lenguaje Natural - Aitana Villaplana Moreno
En esta práctica sen va a desarrollar diferentes sistemas para realizar una tarea de análisis de sentimientos, se desarrollarán con modelos de transformer.

La práctica consistirá en probar variaciones de distintos modelos de transformers, probando también a utilizarlos pre-entrenados, re-entrenados para la tarea concreta de análisis de sentimientos, y ajustados para esta tarea en cuestión. También se realizará un análisis de los resultados, y la variación entre los distintos modelos.

##1. Importación de librerías y carga de los datos
En primer lugar se importarán las librerías necesarias para el desarrollo de la práctica. En segundo lugar se cargarán los dos ficheros del conjunto de datos, subidos la carpeta del drive, el conjunto de entrenamiento (training) y el conjunto de validación (test).

In [ ]:
!pip install --upgrade pip

import pandas as pd

!pip install datasets
import datasets
from torch.utils.data import DataLoader

# Transformers
!!pip install transformers[sentencepiece]
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Se carga el dataset en un objeto de la clase datasets, para posteriormente utilizarlo con los modelos de huggingface.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# base_url = "/content/drive/My Drive/"
base_url = "/home/datasets/"
dataset = datasets.load_dataset('csv', data_files={'train': base_url + 'train_reviews.csv', 'test': base_url + 'test_reviews.csv'})

Using custom data configuration default-8bf5e8f4adbdee32


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8bf5e8f4adbdee32/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## 2. Pre-procesado de datos y representación
Cada modelo de transformer, tiene una capa específica encargada de tokenizar el texto, y prepararlo para introducir en el modelo, por lo que se necesita cargar dicha capa para cada modelo que se vaya a utilizar.

La función `tokenize` sirve para realizar la tokenización con la capa correspondiente de tokenizar el modelo que se vaya a utilizar. Así como también realiza el procesado de las etiquetas, convirtiendolas a números, y eliminando el texto del dataset, ya que el modelo necesita exclusivamente datos numéricos.

In [ ]:
def tokenize(dataset, tokenizer):
    labels = datasets.ClassLabel(num_classes=2, names=["negative", "positive"])

    def tokenize_function(dataset):
        tokens = tokenizer(dataset["review"], padding="max_length", truncation=True)
        tokens["labels"] = labels.str2int(dataset["sentiment"])
        return tokens

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.remove_columns(["id"])
    tokenized_datasets = tokenized_datasets.remove_columns(["review"])
    tokenized_datasets = tokenized_datasets.remove_columns(["sentiment"])
    tokenized_datasets.set_format("torch")
    return tokenized_datasets

La función `data_loader` tranforma el dataset original en dos objetos iterables de tipo DataLoader, para entrenamiento y test respectivamente, ya que es necesario tener los datasets en este tipo de objeto para poder re-entrenar y evaluar el modelo con PyTorch.
Para evitar problemas de memoria y lentitud en el procesamiento, se redujo el dataset original a 1000 reviews, y el tamaño del batch a 16.

In [ ]:
def data_loader(dataset, tokenizer, batch_size=16):
    tokenized_datasets = tokenize(dataset, tokenizer)

    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

    train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=batch_size)
    test_dataloader = DataLoader(small_eval_dataset, batch_size=batch_size)
    return train_dataloader, test_dataloader

## 3. Modelos de Transformers

Una vez realizado el pre-procesamiento, se puede empezar a pensar en un modelo apropiado para realizar la tarea de Clasificación. La arquitectura transformer esta formada por dos partes, el encoder y el decoder. El encoder recibe una secuencia de texto y genera una representación que pasaría a ser utilizada por el decoder. Dicho decoder utiliza como entrada la salida del encoder (y podría utilizar alguna otra entrada) y genera una salida (que puede ser una predicción).

La diferencia de estas redes con las redes recurrentes (RNN) es que en los transformers se tiene en cuenta la relevancia de las palabras en la frase a la hora de tratarlas (la atención), cosa que no se hace en las RNN, por lo que se las palabras más importantes tienen más atención.

Dichos modelos se pueden utilizar pre-entrenados en conjuntos de datos grandes, para posteriormente afinarlo para la tarea / dataset que se pretende utilizar.

Se presentarán los siguientes enfoques:

*   Modelo pre-entrenado generalista ajustado a la colección de entrenamiento.
*   Modelo ajustado a una tarea de análisis de sentimiento con otro dataset.
*   Modelo pre-entrenado para la tarea específica ajustado a la colección de entrenamiento.

Éstos enfoques se probarán con uno o más modelos distintos de transformer.

Existen varias formas de realizar estos re-entrenamientos según la documentación oficial: https://huggingface.co/docs/transformers/training, en este caso se ha decidido utilizar PyTorch nativo para poder entrar más en detalles a la hora del entrenamiento.

Primero se definen las epoch, en este caso es 1 para ahorrar tiempo de entrenamiento.
La función `load_params` crea la función de optimizador en función de los parámetros el modelo, y se inicializan el numero de pasos de entrenamiento para el bucle de entrenamiento posterior.
También se crea un learning rate scheduler para que el lr varíe dinámicamente durante el entrenamiento.

En caso de utilizar una NVIDIA GPU se puede entrenar el modelo en ésta, en este caso las pruebas se han hecho sobre una CPU.

In [ ]:
from transformers import get_scheduler
from torch.optim import AdamW
import torch

epochs = 1
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def load_params(model, train_dataloader):
    optimizer = AdamW(model.parameters(), lr=5e-5)
    num_training_steps = epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
    )
    return optimizer, num_training_steps, lr_scheduler

La función `train_model` se encarga del bucle de entrenamiento del modelo.

In [ ]:
def train_model(model, device, num_training_steps, train_dataloader, optimizer, lr_scheduler):
    from tqdm.auto import tqdm

    progress_bar = tqdm(range(num_training_steps))

    model.train()
    for epoch in range(epochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

La función `evaluate_model` se encarga del bucle de evaluación del modelo, en este caso, con las métricas accuracy, precision, recall, f1score.

In [ ]:
def evaluate_model(model, test_dataloader):
    from tqdm.auto import tqdm

    progress_bar = tqdm(range(len(test_dataloader)))

    accuracy = datasets.load_metric("accuracy")
    precision = datasets.load_metric("precision")
    recall = datasets.load_metric("recall")
    f1score = datasets.load_metric("f1")

    model.eval()
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        accuracy.add_batch(predictions=predictions, references=batch["labels"])
        precision.add_batch(predictions=predictions, references=batch["labels"])
        recall.add_batch(predictions=predictions, references=batch["labels"])
        f1score.add_batch(predictions=predictions, references=batch["labels"])

        progress_bar.update(1)

    print("Accuracy: ", accuracy.compute())
    print("Precision: ", precision.compute())
    print("Recall: ", recall.compute())
    print("F1score: ", f1score.compute())

### 3.1. Modelo pre-entrenado generalista ajustado a la colección de entrenamiento.

Como primera prueba se va a probar a utilizar un modelo general pre-entrenado, el modelo DistilBERT base. Se descargan las dos capas, el tokenizador y el modelo.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

Se crean los iteradores del dataset, y los parámetros para el modelo.


In [ ]:
train_dataloader, test_dataloader = data_loader(dataset, tokenizer)
optimizer, num_training_steps, lr_scheduler = load_params(model, train_dataloader)

  0%|          | 0/8 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-6d33c6302d787d17/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-2baf4a2ff566e188.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-6d33c6302d787d17/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-23c0df40150547f8.arrow


Como el modelo es de uso general, es necesario entrenarlo para la tarea y los datos que se van a utilizar. Posteriormente se guarda el modelo entrenado para posterior uso si fuera necesario.

In [ ]:
train_model(model, device, num_training_steps, train_dataloader, optimizer, lr_scheduler)
torch.save(model, "/content/drive/My Drive/model_1")

  0%|          | 0/63 [00:00<?, ?it/s]

Se evalúa el modelo.

In [ ]:
model = torch.load("/content/drive/My Drive/model_1")
evaluate_model(model, test_dataloader)

  0%|          | 0/63 [00:00<?, ?it/s]

Accuracy:  {'accuracy': 0.882}
Precision:  {'precision': 0.8699186991869918}
Recall:  {'recall': 0.8879668049792531}
F1score:  {'f1': 0.8788501026694046}


Como se puede observar, tiene unos resultados bastante buenos, un 87% de f1 y un 88% de accuracy. Teniendo en cuenta que el modelo pre-entrenado es de uso general y solo se ha entrenado con nuestro dataset, son unos resultados muy buenos.

### 3.2. Modelo ajustado a una tarea de análisis de sentimiento con otro dataset. Transfer Learning

Para esta segunda prueba se va a utilizar un modelo ya previamente ajustado para la tarea se sentiment analysis, pero no se va a realizar el ajusta para 
este dataset en concreto, se va a realizar una prueba de transfer learning.

Cuando se utiliza un modelo ya ajustado para esta tarea específica, hay que tener en cuenta que es un problema de clasificación, y el modelo habrá sido entrenado para un número *N* de clases, por lo que en este caso es necesario utilizar uno entrenado con 2 clases (positivo y negativo).


#### 3.2.1 SiEBERT - English-Language Sentiment Classification

Se va a realizar una primer prueba con el modelo https://huggingface.co/siebert/sentiment-roberta-large-english, entrenado a partir del RoBERTa-large, para la tarea de sentiment analysis. Se utilizaron múltiples fuentes para entrenar el modelo, entre ellas reviews y tweets.

Se trata de un modelo pesado, ya que es un fine-tuning de un modelo large, por lo que fue necesario el uso de una VM de Google Cloud Platform para poder entrenarlo con mayores recursos (26GB de RAM en este caso).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english", num_labels=2)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

En esta ocasión cargamos los datos, pero no entrenamos el modelo.

In [ ]:
train_dataloader, test_dataloader = data_loader(dataset, tokenizer)

  0%|          | 0/8 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-6d33c6302d787d17/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-8a0a43dc690c2848.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-6d33c6302d787d17/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3cb1b475e2ecf80d.arrow


Directamente pasamos a evaluarlo con el conjunto de test, teniendo en cuenta que se tienen las mismas labels en los datos y en el modelo pre-entrenado.

In [ ]:
evaluate_model(model, test_dataloader)

  0%|          | 0/63 [00:00<?, ?it/s]

Accuracy:  {'accuracy': 0.947}
Precision:  {'precision': 0.9479768786127167}
Recall:  {'recall': 0.9498069498069498}
F1score:  {'f1': 0.948891031822565}


Se puede ver que los resultados son muy buenos, con un 95% de f1 y accuracy, mucho mejores a los obtenidos con el dataset general, a pesar de no haber entrenado este modelo, como el modelo estaba previamente entrenado para esta tarea, los resultados han sido muy buenos.

#### 3.2.2 CamemBERT fine-tuned for Movie Review Sentiment Anlalysis 

Para la segunda prueba se va a utilizar el modelo https://huggingface.co/mrm8488/camembert-base-finetuned-movie-review-sentiment-analysis, entrenado a partir del transformer CamemBERT, para la tarea de sentiment analysis de review de películas, por lo que puede ser apropiado para esta tarea ya que los datos son también reviews de películas.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/camembert-base-finetuned-movie-review-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/camembert-base-finetuned-movie-review-sentiment-analysis", num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/455 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/299 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/745 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [ ]:
train_dataloader, test_dataloader = data_loader(dataset, tokenizer)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
evaluate_model(model, test_dataloader)

  0%|          | 0/63 [00:00<?, ?it/s]

Accuracy:  {'accuracy': 0.783}
Precision:  {'precision': 0.7635726795096323}
Recall:  {'recall': 0.8416988416988417}
F1score:  {'f1': 0.8007346189164373}


En esta ocasión los resultados han sido buenos, 80% f1 y 78% accuracy, pero no tan buenos como en la prueba anterior. Esto es posiblemente debido a que es un modelo más pequeño, y no tan completo como el anterior. También es posible que al haberse entrenado solo con reviews de películas, se haya producido un sobre-entrenamiento, y el modelo no sea tan capaz de generalizar, como en el caso anterior, que tenía más variedad de datos de entrenamiento. 

### 3.3. Modelo pre-entrenado para la tarea específica ajustado a la colección de entrenamiento.

Una vez probado un modelo general re-entreando para el dataset específico, y un modelo fine-tuneado para la tarea, pero no re-entrenado, se va a realizar una tercera prueba, utilizando un modelo ya previamente ajustado para la tarea se sentiment analysis, pero en este caso si se realizará un re-entrenamiento para ajustar el modelo con los datos de este problema. 

Así es posible realizar comparaciones y ver si vale la pena gastar tiempo y recursos en ajustarlo.

#### 3.3.1 SiEBERT - English-Language Sentiment Classification (ajustado) 

Primero se realiza el entrenamiento con el primer modelo probado en el apartado 3.2. Para realizar este entrenamiento, fue necesario conectar una VM de GCP con 52GB de RAM, ya que el modelo es bastante pesado.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english", num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [ ]:
train_dataloader, test_dataloader = data_loader(dataset, tokenizer)
optimizer, num_training_steps, lr_scheduler = load_params(model, train_dataloader)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
train_model(model, device, num_training_steps, train_dataloader, optimizer, lr_scheduler)
torch.save(model, "/content/drive/My Drive/model_3")

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
model = torch.load("/content/drive/My Drive/model_3")
evaluate_model(model, test_dataloader)

  0%|          | 0/63 [00:00<?, ?it/s]

Accuracy:  {'accuracy': 0.955}
Precision:  {'precision': 0.9592233009708738}
Recall:  {'recall': 0.9536679536679536}
F1score:  {'f1': 0.9564375605033882}


Como se puede ver, los resultados a penas varía respecto al modelo sin ajustar a este dataset. Es posible que los datos aportados en este nuevo entrenamiento ya sean redundantes ya que es un modelo grande y entrenado con gran cantidad de datos.

También es cierto que el modelo ya presentaba unos resultados casi inmejorables, dificilmente superables.

#### 3.3.2 CamemBERT fine-tuned for Movie Review Sentiment Anlalysis (ajustado)

En segundo lugar se realiza el entrenamiento con el segundo modelo probado. Para este entrenamiento también se utilizó una VM de GCP con 52GB de RAM.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/camembert-base-finetuned-movie-review-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/camembert-base-finetuned-movie-review-sentiment-analysis", num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/455 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/299 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/745 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [ ]:
train_dataloader, test_dataloader = data_loader(dataset, tokenizer)
optimizer, num_training_steps, lr_scheduler = load_params(model, train_dataloader)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
train_model(model, device, num_training_steps, train_dataloader, optimizer, lr_scheduler)
torch.save(model, "/content/drive/My Drive/model_4")

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
model = torch.load("/content/drive/My Drive/model_4")
evaluate_model(model, test_dataloader)

  0%|          | 0/63 [00:00<?, ?it/s]

Accuracy:  {'accuracy': 0.518}
Precision:  {'precision': 0.518}
Recall:  {'recall': 1.0}
F1score:  {'f1': 0.6824769433465085}


En esta ocasión se puede ver que los resultados son bastante peores que en el caso del modelo sin ajustar. Esto puede ser debido a que se haya producido un sobre ajuste, ya que los datos con los que el modelo fue entrenado son ya bastante específicos. 

El mayor indicativo de este sobre-entrenamiento es que el tenemos un 100% de recall (se han detectado todos los casos de positivo o negativo), pero una baja precisión (se han dado casos positivos por negativos y o la inversa). Este tipo de escenario normalmente se da cuando hay desbalance en los datos, aunque en este caso el dataset que se está utilizando esta balanceado, pero puede que el modelo ya tuviera un desbalance anterior y se haya terminado de desbalancear.

## 4. Comparativa entre modelos pre-entrenados y ajustados

En este apartado se va sumarizar todos los resultados obtenidos en las pruebas realizadas.

*   **distilbert-base-uncased (ajustado)**

    *   Arquitectura: 6-layer, 768-hidden, 12-heads, 66M parameters
    *   Tiempo: 45 minutos entrenamiento / 15 minutos evaluación
    *   F1score: 88%

*   **SiEBERT (sin ajustar)**, viene de RoBERTa-large

    *   *Nota: Fue necesario realizar la evaluación con una MV de 26GB de RAM y una GPU de 4 cores*
    *   Arquitectura: 24-layer, 1024-hidden, 16-heads, 355M
    *   Tiempo: Sin entrenamiento / 1h 30 min evaluación
    *   F1score: 95%

*   **CamemBERT fine-tuned (sin ajustar)**, viene de CamemBERT

    *   Arquitectura: 12-layer, 768-hidden, 12-heads, 110M parameters
    *   Tiempo: Sin entrenamiento / 30 min evaluación
    *   F1score: 80% 

*   **SiEBERT (ajustado)**, viene de RoBERTa-large

    *   *Nota: Fue necesario el entrenamiento con una MV de 52GB de RAM y una GPU de 8 cores*
    *   Arquitectura: 24-layer, 1024-hidden, 16-heads, 355M
    *   Tiempo: 1h entrenamiento / 20 minutos evaluación
    *   F1score: 95%

*   **CamemBERT fine-tuned (ajustado)**, viene de CamemBERT

    *   *Nota: Fue necesario el entrenamiento con una MV de 52GB de RAM y una GPU de 8 cores*
    *   Arquitectura: 12-layer, 768-hidden, 12-heads, 110M parameters
    *   Tiempo: 18 minutos entrenamiento / 6 minutos evaluación
    *   F1score: 68%


Es importante tener en cuenta los recursos que se necesitaron para entrenado y/o evaluar los modelos, lo que hace que el tiempo suba o baje. El número de parámetros del modelo, también es directamente prporcional al tiempo y espacio utilizados.

Por ejemplo, el modelo de CamemBERT se pudo evaluar con el entorno normal de Google Colab, pero para el entrenamiento se necesitó una máquina más potente, lo que redujo considerablemente el tiempo de evaluación una vez entrenado.

## 5. Conclusiones

Como conclusión del trabajo, se puede ver que los modelos de transformer funcionan bastante bien, sin tener que dedicar demasiados esfuerzos a ajustarlos. Se ha visto que un modelo general, como distilBERT, ajustado al dataset de la tarea en cuestión, da muy buenos resultados a pesar de ser un modelo relativamente ligero, y rápido de entrenar.

También hemos visto que si se quieren mejores resultados, una buena idea es utilizar modelo más pesados, ya entrenados para la misma tarea que se quiere realizar (como SiEBERT) funcionan muy bien, aunque no se ajusten al dataset específico, de hecho, en esta ocasión, no mejoran los resultados aún ajustandolo con el dataset en cuestión.

A pesar de esto, no todos los modelos ajustados a la tarea funcionan tan bien, y puede incluso empeorar con el entrenamiento, como en el caso del segundo modelo (CamemBERT fine-tuned), por lo que hay que escoger cuidadosamente que modelos se utilizan para transfer learning, y para re-entrenar.